In [1]:
#baselines
PREDICTION_FILE = "result_baseline_doc2vec.csv"
#PREDICTION_FILE = "result_svm.csv"
#proposed methods
#PREDICTION_FILE = "result_stm.csv"
#PREDICTION_FILE = "result_stm_all.csv"
#PREDICTION_FILE = "result_stm_threshold.csv"
#PREDICTION_FILE = "result_stm_viterbi.csv"
#PREDICTION_FILE = "result_stm_oracle.csv"

In [2]:
iab_labels = ["arts-entertainment", "automotive", "business", "careers", "education", "family-parenting", 
          "health-fitness", "food-drink", "hobbies-interests", "home-garden", "law-gov-t-politics", 
          "news", "personal-finance", "society", "science", "pets", "sports", "style-fashion", 
          "technology-computing", "travel", "real-estate", "shopping", "religion-spirituality"]
label_map = dict()

for i in range(23):
    label_map[iab_labels[i]] = "IAB" + str(i+1)
    print("IAB" + str(i+1), "\t", iab_labels[i])

IAB1 	 arts-entertainment
IAB2 	 automotive
IAB3 	 business
IAB4 	 careers
IAB5 	 education
IAB6 	 family-parenting
IAB7 	 health-fitness
IAB8 	 food-drink
IAB9 	 hobbies-interests
IAB10 	 home-garden
IAB11 	 law-gov-t-politics
IAB12 	 news
IAB13 	 personal-finance
IAB14 	 society
IAB15 	 science
IAB16 	 pets
IAB17 	 sports
IAB18 	 style-fashion
IAB19 	 technology-computing
IAB20 	 travel
IAB21 	 real-estate
IAB22 	 shopping
IAB23 	 religion-spirituality


## Micro F1 & Macro F1

In [3]:
from collections import Counter
import numpy as np

with open(PREDICTION_FILE, "r") as f:
    tps, fps, fns = 0, 0, 0  # stats for microF1
    label_metrics = dict()  # stats for macroF1
    for line in f:
        line = line.strip()
        label_file, prediction = line.split(",")
        with open("label/" + label_file, "r") as f_label:
            label = f_label.readline().strip()
            if label == label_map['news']:
                continue
        predictions = prediction.split(" ")
        
        if PREDICTION_FILE.startswith("result_stm") or PREDICTION_FILE.startswith("result_baseline"):
            predictions = [label_map[pred] for pred in predictions]
        
        labels = label.split(" ")
        if label_map['news'] in predictions: 
            predictions.remove(label_map['news'])
        if label_map['news'] in labels: 
            labels.remove(label_map['news'])
            
        # update micro F1 stats
        tp = len(set(predictions).intersection(labels))
        fp = len(predictions) - tp
        fn = len(labels) - tp
        tps += tp
        fps += fp
        fns += fn
        
        # update macro F1 stats
        total_labels = set(predictions + labels)
        for l in total_labels:
            if not l in label_metrics:
                label_metrics[l] = Counter()
            if l in predictions and l in labels:
                label_metrics[l].update(["tp"])
            elif l in predictions and l not in labels:
                label_metrics[l].update(["fp"])
            elif l not in predictions and l in labels:
                label_metrics[l].update(["fn"])
        
    micro_f1 = 2*tps*1.0/(2*tps + fps + fns)
    print("micro_f1:", micro_f1)
    print("tp:", tps, "fp:", fps, "fn:", fns)
    
    macro_list = []
    for label, c in label_metrics.items():
        f1 = 2*c['tp']*1.0/(2*c['tp'] + c['fp'] + c['fn'])
        # print(label, f1)
        macro_list.append(f1)
    # this line is wrong since the items in python list aren't ordered    
    print(dict(zip(iab_labels, macro_list)))
    a = np.array(macro_list)
    print("macro_f1: ", np.mean(a))

micro_f1: 0.46674119619899385
tp: 835 fp: 665 fn: 1243
{'arts-entertainment': 0.4634146341463415, 'automotive': 0.2185792349726776, 'business': 0.41025641025641024, 'careers': 0.6265060240963856, 'education': 0.6666666666666666, 'family-parenting': 0.10852713178294573, 'health-fitness': 0.5882352941176471, 'food-drink': 0.5602605863192183, 'hobbies-interests': 0.10344827586206896, 'home-garden': 0.7766990291262136, 'law-gov-t-politics': 0.40540540540540543, 'news': 0.42016806722689076, 'personal-finance': 0.3252032520325203, 'society': 0.4803921568627451, 'science': 0.3086053412462908, 'pets': 0.03225806451612903, 'sports': 0.7020408163265306, 'style-fashion': 0.6813186813186813, 'technology-computing': 0.4927536231884058, 'travel': 0.390625, 'real-estate': 0.4918032786885246, 'shopping': 0.075}
macro_f1:  0.4240075897344862


## Evaluate SVM 
The prediction file format is a bit different. Need special handling

In [57]:
SVM_PREDICTION_FILE = "result_json_svm.txt"
FILE_ORDER = "filename_order_svm.txt"

In [29]:
svm_labels = ["Art-entertainment", "Automotive", "Business", "Career", "Education", "Family-parenting", 
          "Health-fitness", "Food-drink", "Hobby-interest", "Home-garden", "Law-gov-politics", 
          "News", "Personal-finance", "Society", "Science", "Pet", "Sport", "Style-fashion", 
          "Technology", "Travel", "Real-estate", "Shopping", "Religion-spirituality"]
svm_label_map = dict()

for i in range(23):
    svm_label_map[svm_labels[i]] = "IAB" + str(i+1)
    print(svm_labels[i], '\t', "IAB" + str(i+1))

Art-entertainment 	 IAB1
Automotive 	 IAB2
Business 	 IAB3
Career 	 IAB4
Education 	 IAB5
Family-parenting 	 IAB6
Health-fitness 	 IAB7
Food-drink 	 IAB8
Hobby-interest 	 IAB9
Home-garden 	 IAB10
Law-gov-politics 	 IAB11
News 	 IAB12
Personal-finance 	 IAB13
Society 	 IAB14
Science 	 IAB15
Pet 	 IAB16
Sport 	 IAB17
Style-fashion 	 IAB18
Technology 	 IAB19
Travel 	 IAB20
Real-estate 	 IAB21
Shopping 	 IAB22
Religion-spirituality 	 IAB23


In [59]:
label_files = []

with open(FILE_ORDER, "r") as f:
    for line in f:
        line = line.strip()
        label_files.append(line)
        
print(len(label_files))

1578


In [ ]:
from collections import Counter
import numpy as np
import json

with open(SVM_PREDICTION_FILE, "r") as f:
    tps, fps, fns = 0, 0, 0  # stats for microF1
    label_metrics = dict()  # stats for macroF1
    doc_id = 0
    
    f_w = open("result_svm.csv", "w+")
    for line in f:
        line = line.strip()
        label_file = label_files[doc_id]
        doc_id +=1
        if label_file == '.DS_Store':
            continue
            
        with open("label/" + label_file, "r") as f_label:
            label = f_label.readline().strip()
            if label == label_map['news']:
                continue
                
        prediction_json = json.loads(line)
        predictions = set()
        if 'taxonomy' not in prediction_json:
            print(prediction_json)
            continue
        for pred in prediction_json['taxonomy']:
            label = pred['label']
            if "/" in label:
                label = label[:label.index("/")]
            if label == "Others":
                continue
            score = pred['score']
            if score > 0:
                predictions.add(svm_label_map[label])
        predictions = list(predictions)
        f_w.write(label_file)
        f_w.write(",")
        f_w.write(" ".join(predictions))
        f_w.write("\n")
    f_w.close()
        